# PyTorch Seq2Seq 챗봇 구현 및 데이터 활용

In [1]:
import os, sys
sys.path.append(os.pardir)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim
import re
import core.config as conf
from konlpy.tag import Mecab  # tweepy오류로 konlpy 직접 설치 필요, mecab별도 설치 readme 참고

가상 환경 생성<br> 
`conda create -n 가상환경이름 python=3.7`

PyTorch(Windows, Conda, CUDA 10.2)<br>
`conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch`

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Field, BucketIterator

PyTorch에서 GPU가 사용가능한지 확인

In [3]:
torch.cuda.is_available() # GPU 사용 여부
torch.cuda.get_device_name(0) # GPU 정보Z
torch.cuda.device_count() # 사용가능 GPU 개수

1

In [4]:
# CUDA_VISIBLE_DEVICES=0 # Device Setting

# 데이터 전처리를 위한 설정
Seq2Seq에서의 임베딩은 아래와 같이 추가 토큰을 사용하여 동작을 제어한다.

- `<PAD>`: 0, Padding, 짧은 문장을 채울 때 사용하는 토큰
- `<SOS>`: 1, Start of Sentence, 문장의 시작을 나타내는 토큰
- `<EOS>`: 2, End of Sentence, 문장의 끝을 나타내는 토큰
- `<UNK>`: 3, Unkown Words, 없는 단어를 나타내는 토큰

디코더 입력에 <SOS>가 들어가면 디코딩(문장)의 시작을 의미하고 출력에 <EOS>가 나오면 디코딩(문장)을 종료한다.
<br>

In [120]:
# special tokens
pad = "<PAD>"# 패딩
sos = "<SOS>"# 시작
eos = "<EOS>"# 끝
unk = "<UNK>"# Unkown word

# 태그 인덱스
PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3

# 데이터 불러오기

In [121]:
path = conf.data_path
data_df = pd.read_csv(f'{path}'+'ChatbotData.csv', encoding='utf-8')
data_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [122]:
len(data_df)

11823

In [123]:
question, answer = list(data_df['Q']), list(data_df['A'])

# 형태소분석

챗봇 데이터 문장을 먼저 최소단위로 tokenizing 해야한다. 한국어는 보통 KoNLPy를 사용한다.

해당 예시에서는 mecab 사용

In [124]:
RE_FILTER = re.compile("[.,!?\"':;~()]") # 정규 표현식 필터

def pos_tag(sentences):
    mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic") # KoNLPy mecab 사용
    # 일반적인 Konlpy의 토크나이저와는 달리, mecab은 dicpath 파라미터를 지정 필요
    
    sentences_pos = []     # 문장 품사 변수 초기화
    # 모든 문장 반복
    for s in sentences:
        s = re.sub(RE_FILTER, "", s)  # 특수기호 제거
        s = " ".join(mecab.morphs(s))  # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentences_pos.append(s)
        
    return sentences_pos

In [125]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

In [126]:
# 형태소분석을 수행한 데이터 출력
for i in range(5):
    print('Q : %s \nA: %s\n'%(question[i], answer[i]))

Q : 12 시 땡 
A: 하루 가 또 가 네요

Q : 1 지망 학교 떨어졌 어 
A: 위로 해 드립니다

Q : 3 박 4 일 놀 러 가 고 싶 다 
A: 여행 은 언제나 좋 죠

Q : 3 박 4 일 정도 놀 러 가 고 싶 다 
A: 여행 은 언제나 좋 죠

Q : PPL 심하 네 
A: 눈살 이 찌푸려 지 죠



질문과 대답 문장들을 모두 합쳐서 전체 단어 사전을 만든다. 단어를 인덱스에 따라 정리를 해야 단어와 인덱스간의 전환이 용이하다.

- 문장 → 인덱스 배열 임베딩 레이어의 입력으로 사용
- 모델의 출력(인덱스 배열) → 문장


### 딕셔너리 생성
단어 → 인덱스: 모델 입력으로는 인덱스가 필요, 문장을 인덱스로 변환할 때 사용가능<br>
인덱스 → 단어: 모델의 예측 결과는 인덱스로 출력, 인덱스를 문장으로 변환할 때  사용

In [127]:
sentences = question + answer # 질문과 대답 문장들을 하나로 합침
vocab = []

for s in sentences:
    for word in s.split():
        if len(word) > 0: # 길이가 0인 단어는 제외
            vocab.append(word) # 전체 단어로 딕셔너리 생성
            
vocab = list(set(words)) # 중복 단어 삭제
vocab[:0] = [pad, sos, eos, unk] # 제일 앞에 태그 단어 삽입

In [128]:
len(vocab) # vocab 단어개수

6809

In [129]:
word_to_index = {word: index for index, word in enumerate(vocab)} # 단어 → 인덱스
index_to_word = {index: word for index, word in enumerate(vocab)} # 인덱스 → 단어

In [130]:
dict(list(word_to_index.items())[:10])

{'<PAD>': 1331,
 '<SOS>': 1138,
 '<EOS>': 3098,
 '<UNK>': 3,
 '여친': 4,
 '상종': 5,
 '믿음': 6,
 '물론': 7,
 '예의': 8,
 '힘드실': 9}

In [131]:
dict(list(index_to_word.items())[:10])

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: '여친',
 5: '상종',
 6: '믿음',
 7: '물론',
 8: '예의',
 9: '힘드실'}

In [134]:
# 문장을 인덱스로 변환
def Sent2Idx(sentences, vocabulary, type): 
    result = []
    
    for sentence in sentences:     # 모든 문장에 대해서 반복
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 SOS 태그 추가
        if type == DECODER_INPUT:
            result.extend([vocabulary[sos]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                sentence_index.extend([vocabulary[word]]) # 사전에 있는 단어면 해당 인덱스를 추가
            else:
                sentence_index.extend([vocabulary[unk]]) # 사전에 없는 단어면 UNK 인덱스를 추가

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 eos 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[eos]]
            else:
                sentence_index += [vocabulary[eos]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[pad]]         # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        result.append(sentence_index)         # 문장의 인덱스 배열을 추가

    return np.asarray(sentences_index)

# 인덱스를 문장으로 변환
def Idx2Sent(indexs, vocab): 
    sentence = ''
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == EOS_IDX:
            break; # 종료 인덱스면 중지
        if vocabulary.get(index) is not None:
            sentence += vocab[index] # 사전에 있는 인덱스면 해당 단어를 추가
        else:
            sentence.extend([vocab[UNK_IDX]])# 사전에 없는 인덱스면 unk 단어를 추가
            
        sentence += ' '  # 빈칸 추가

    return sentence

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요하다..

인코더 입력 : 질문<br>
디코더 입력 : <sos> 답변 <br>
디코더 출력 : 답변 <eos>

In [133]:
# 인코더 입력 인덱스 변환
Sent2Idx(question, word_to_index, ENCODER_INPUT)
Sent2Idx(answer, word_to_index, DECODER_INPUT)

NameError: name 'sentences_index' is not defined

# 모델 정의
Seq2seq 모델은 가변 길이 시퀀스를 입력으로 받고, 크기가 고정된 모델을 이용하여, 가변 길이 시퀀스를 출력으로 반환하는 것을 목표로 한다.

- Encoder(인코더): 가변 길이 입력 시퀀스를 고정 길이의 Context Vector로 인코딩
- Context Vector(문맥 벡터): RNN의 마지막 은닉 레이어로 입력 시퀀스의 의미론적 정보
- Decoder(디코더): 단어 하나, 문맥 벡터를 입력으로 받고, 시퀀스의 다음 단어가 무엇일지 추론한 결과와 단계에서 사용할 은닉 상태를 반환

## 1. Encoder
Encoder는 양방향 GRU(Bidirectional-Gated Recurrent Unit)를 사용하여 2개층으로 구성되어있다.입력 데이터의 Context 정보를 고정 길이의 벡터로 바꿔주는 역할을 하는데, 다시 말해 시퀀스의 요약된 정보를 Decoder로 전달하는 것이 목표라고 할 수 있다. 

In [113]:
# Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers         # embedding: 입력값을 emd_dim 벡터로 변경
        self.embedding = nn.Embedding(input_dim, emb_dim)         # embedding을 입력받아 hid_dim 크기의 hidden state, cell 출력
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # sre: [src_len, batch_size]
        embedded = self.dropout(self.embedding(src))  # initial hidden state는 zero tensor
        outputs, (hidden, cell) = self.rnn(embedded)
        # output: [src_len, batch_size, hid dim * n directions]
        # hidden: [n layers * n directions, batch_size, hid dim]
        # cell: [n layers * n directions, batch_size, hid dim]

        return hidden, cell

## 파라미터 설명

- input_dim = input 데이터의 vocab size = one-hot vector의 사이즈, 단어들의 index가 embedding 함수로 넘겨짐
- emb_dim = embedding layer의 차원
  (embedding 함수 : one-hot vector를 emb_dim 길이의 dense vector로 변환)
- hid_dim = 은닉 상태의 차원 ( = cell state의 차원)
- n_layers = RNN 안의 레이어 개수 (여기선 2개)
- dropout = 사용할 드롭아웃의 양 (오버피팅 방지하는 정규화 방법)
- n_directions = 1  (cf. bidirectional RNN의 경우 : n_directions=2)

- 초기 은닉 상태, cell state 명시해주지 않으면, 디폴트로 모두 0으로 채워진 텐서로 초기화

- outputs : 맨 위 레이어에서 각 time-stamp마다의 은닉 상태들

- hidden : 각 레이어의 마지막 은닉상태, h_T

- cell : 각 레이어의 마지막 cell state, c_T

## 2. 디코더(Decoder)

Decoder는 Encoder가 전달한 Context Vector를 전달받아 결과를 도출한다. 

- Layer 1 : 직전 hidden state, cell state, embedded token 입력으로 받아 새로운 hidden stat와 cell state를 만들어냄 

- Layer 2 : Layer 1의 은닉 상태, Layer 2에서 직전 hidden state, cell state를 입력fm로 받아 새로운 은닉 hidden state와 cell state를 만들어냄

    - *Encoder Layer 1의 마지막 hidden state, cell state = context vector = Decoder Layer1 첫 hidden state, cell state*

- Decoder RNN/LSTM의 맨 위 Layer의 은닉 상태를 Linear Layer에 넘겨서 다음 토큰을 예측함

In [114]:
# decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # content vector를 입력받아 emb_dim 출력
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # embedding을 입력받아 hid_dim 크기의 hidden state, cell 출력
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input: [batch_size]
        # hidden: [n layers * n directions, batch_size, hid dim]
        # cell: [n layers * n directions, batch_size, hid dim]

        input = input.unsqueeze(0) # input: [1, batch_size], 첫번째 input은 <SOS>

        embedded = self.dropout(self.embedding(input)) # [1, batch_size, emd dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [seq len, batch_size, hid dim * n directions]
        # hidden: [n layers * n directions, batch size, hid dim]
        # cell: [n layers * n directions, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0)) # [batch size, output dim]
        
        return prediction, hidden, cell

## 파라미터 설명

- output_dim = output 데이터의 vocab size (cf. input_dim은 데이터에서 주어진대로, output_dim은 우리가 직접 정해서 초기화)
- hidden, cell은 각 time-stamp/각 레이어들의 은닉상태와 cell state들의 리스트
- output은 마지막 time-stamp/마지막 레이어의 은닉상태만 
- input = [batch size] → unsqueeze → input = [1, batch size]
- input = [1, batch size] → embedding → dropout → embedded = [1, batch size, emb dim] 
- embedded, hidden, cell → rnn → output, hidden, cell

- output = [1, batch size, hid dim] → squeeze → output = [batch size, hid dim]
- output = [batch size, hid dim] → fc_out → prediction = [batch size, output dim]

In [46]:
# Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.hid_dim == decoder.hid_dim, \
            'Hidden dimensions of encoder decoder must be equal'
        # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.n_layers == decoder.n_layers, \
            'Encoder and decoder must have equal number of layers'

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [src len, batch size]
        # trg: [trg len, batch size]
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0] # 타겟 토큰 길이 얻기
        trg_vocab_size = self.decoder.output_dim # context vector의 차원

        # decoder의 output을 저장하기 위한 tensor
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # initial hidden state
        hidden, cell = self.encoder(src)

        # 첫 번째 입력값 <sos> 토큰
        input = trg[0,:]

        for t in range(1,trg_len): # <eos> 제외하고 trg_len-1 만큼 반복
            output, hidden, cell = self.decoder(input, hidden, cell)

            # prediction 저장
            outputs[t] = output

            # teacher forcing을 사용할지, 말지 결정
            teacher_force = random.random() < teacher_forcing_ratio

            # 가장 높은 확률을 갖은 값 얻기
            top1 = output.argmax(1)

            # teacher forcing의 경우에 다음 lstm에 target token 입력
            input = trg[t] if teacher_force else top1

        return outputs

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.????????????????


# 모델 학습 및 테스트